In [10]:
import argparse
import os
import time

import torch

import pytorch_mask_rcnn as pmr
    
    
def main(args):
    device = torch.device("cuda" if torch.cuda.is_available() and args.use_cuda else "cpu")
    cuda = device.type == "cuda"
    if cuda: pmr.get_gpu_prop(show=True)
    print("\ndevice: {}".format(device))
    
    #d_test = pmr.datasets(args.dataset, args.data_dir, "val", train=True) # VOC 2012. set train=True for eval
    d_test = pmr.datasets(args.dataset, args.data_dir, "val2017", train=True) # COCO 2017

    print(args)
    num_classes = max(d_test.classes) + 1
    model = pmr.maskrcnn_resnet50(False, num_classes).to(device)
    
    checkpoint = torch.load(args.ckpt_path, map_location=device)
    model.load_state_dict(checkpoint["model"])
    #print(checkpoint["eval_info"])
    del checkpoint
    if cuda: torch.cuda.empty_cache()

    print("\nevaluating...\n")
    
    B = time.time()
    eval_output, iter_eval = pmr.evaluate(model, d_test, device, args)
    B = time.time() - B
    
    print(eval_output.get_AP())
    if iter_eval is not None:
        print("\nTotal time of this evaluation: {:.1f} s, speed: {:.1f} imgs/s".format(B, 1 / iter_eval))
    
    
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--dataset", default="coco")
    parser.add_argument("--data-dir", default="/home/kaya/PyTorch-Simple-MaskRCNN/image")
    parser.add_argument("--ckpt-path", default="/home/kaya/PyTorch-Simple-MaskRCNN/maskrcnn_coco-3.pth")
    parser.add_argument("--iters", type=int, default=3) # number of iterations, minus means the entire dataset
    args = parser.parse_args([]) # [] is needed if you're using Jupyter Notebook.
    
    args.use_cuda = False
    args.results = os.path.join(os.path.dirname(args.ckpt_path), "aaskrcnn_results.pth")
    
    main(args)
    
    


device: cpu
loading annotations into memory...
Done (t=0.77s)
creating index...
index created!
Namespace(dataset='coco', data_dir='/home/kaya/PyTorch-Simple-MaskRCNN/image', ckpt_path='/home/kaya/PyTorch-Simple-MaskRCNN/maskrcnn_coco-3.pth', iters=3, use_cuda=False, results='/home/kaya/PyTorch-Simple-MaskRCNN/aaskrcnn_results.pth')

evaluating...



/home/kaya/PyTorch-Simple-MaskRCNN/.venv/lib/python3.10/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


iter: 1325.2, total: 1313.7, model: 1303.0
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.04s).
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.04s).
accumulate: 0.1s
{'bbox AP': 27.2, 'mask AP': 20.1}

Total time of this evaluation: 5.8 s, speed: 0.8 imgs/s
